### Cardiac MRI View Prediction

The following notebook uses a trained neural network to predict the ES phase for 4-chamber and short-axis series in a directory of dicom files. Prior to running this script, ensure you have run "CAP View Prediction" so that the 4-chamber and short-axis series have been identified. Alternatively, you can run the analysis on a single series of images that you have previously identified as 4-chamber or short-axis views. 

First, import the necessary packages to run the analysis. The required libraries are pydicom, pandas, and tensorflow. The original analysis was run using python 3.6.8, pydicom 1.2.2, and tensorflow 2.4.1.

In [1]:
import os
import sys
import pydicom 
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from tqdm import tqdm
import cv2
import tensorflow as tf

print('Python: {}'.format(sys.version))
print('Pydicom: {}'.format(pydicom.__version__))
print('TensorFlow: {}'.format(tf.__version__))

Python: 3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]
Pydicom: 1.2.2
TensorFlow: 2.4.1


Tensorflow can be run with or without GPU support. If GPU support through tensorflow is enabled and available, the following code will display and available GPU.

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

To disable the GPU, run the following code. DO NOT run this cell if you would prefer to use the available GPU (recommended)

In [3]:
# Only use CPU for right now - Set CPU as only available physical device
my_devices = tf.config.experimental.list_physical_devices(device_type='CPU')
tf.config.experimental.set_visible_devices(devices= my_devices, device_type='CPU')

### 1.0 Single Patient
#### Finding SA and 4CH View Series

To start, we will load only the 4-chamber or short-axis views from a folder of dicom files. Importantly, this analysis expects view selection to have been run prior to this notebook. We will load in the series information from the .csv file that was created by the view selection analysis. 

In [4]:
src = '../reports/EXAMPLE_series_predictions.csv'              # path to input csv containing view information
dst = '../reports/'                                    # path to save resulting predictions

# load the csv as a DataFrame using pandas
views = pd.read_csv(src)
views

,Patient ID,Series ID,Series Number,Frames,Series Description,Predicted View,Confidence
0,CHD1055302,2.16.124.113543.6006.99.4489543484581785604,1501,90,flow_bh_ao_sense,OTHER,1.00
1,CHD1055302,2.16.124.113543.6006.99.4545974342301296934,1101,480,sa_sense,SA,1.00
2,CHD1055302,2.16.124.113543.6006.99.4513285645466080530,1401,80,4ch_sense,4CH,1.00
3,CHD1055302,2.16.124.113543.6006.99.4544301574540994342,1201,60,lvot_sense,LVOT,1.00
4,CHD1055302,2.16.124.113543.6006.99.4575318287899584458,701,120,4ch_sense,4CH,1.00
5,CHD1055302,2.16.124.113543.6006.99.4557103526600474423,501,12,bb_-_ax_pa_clear,OTHER,1.00
6,CHD1055302,2.16.124.113543.6006.99.4492460316146916281,901,90,rt_2ch_sense,2CH RT,1.00
7,CHD1055302,2.16.124.113543.6006.99.4569864505174670623,801,180,rvot_sense,RVOT,1.00
8,CHD1055302,2.16.124.113543.6006.99.7493418895009265798,1801,30,lt_pa_sense,SA,0.73
9,CHD1055302,2.16.124.113543.6006.99.4488755540510057828,601,30,lt_2ch_sense,2CH LT,1.00


In [5]:
print('Number of patients: {}'.format(len(views['Patient ID'].unique())))
print('Number of series: {}'.format(len(views['Series ID'].unique())))

Number of patients: 1
Number of series: 10


The available views for single patient can be found using the following:

In [6]:
patients = views['Patient ID'].unique()
print('Available patients: {}'.format(patients))

# select the first patient
patient_df = views.loc[views['Patient ID'] == patients[0]]
patient_df

Available patients: ['CHD1055302']


,Patient ID,Series ID,Series Number,Frames,Series Description,Predicted View,Confidence
0,CHD1055302,2.16.124.113543.6006.99.4489543484581785604,1501,90,flow_bh_ao_sense,OTHER,1.00
1,CHD1055302,2.16.124.113543.6006.99.4545974342301296934,1101,480,sa_sense,SA,1.00
2,CHD1055302,2.16.124.113543.6006.99.4513285645466080530,1401,80,4ch_sense,4CH,1.00
3,CHD1055302,2.16.124.113543.6006.99.4544301574540994342,1201,60,lvot_sense,LVOT,1.00
4,CHD1055302,2.16.124.113543.6006.99.4575318287899584458,701,120,4ch_sense,4CH,1.00
5,CHD1055302,2.16.124.113543.6006.99.4557103526600474423,501,12,bb_-_ax_pa_clear,OTHER,1.00
6,CHD1055302,2.16.124.113543.6006.99.4492460316146916281,901,90,rt_2ch_sense,2CH RT,1.00
7,CHD1055302,2.16.124.113543.6006.99.4569864505174670623,801,180,rvot_sense,RVOT,1.00
8,CHD1055302,2.16.124.113543.6006.99.7493418895009265798,1801,30,lt_pa_sense,SA,0.73
9,CHD1055302,2.16.124.113543.6006.99.4488755540510057828,601,30,lt_2ch_sense,2CH LT,1.00


In [7]:
# select the 4CH and SA views for this patient
input_df = patient_df.loc[(patient_df['Predicted View'].isin(['SA','4CH']))]
input_df

,Patient ID,Series ID,Series Number,Frames,Series Description,Predicted View,Confidence
1,CHD1055302,2.16.124.113543.6006.99.4545974342301296934,1101,480,sa_sense,SA,1.00
2,CHD1055302,2.16.124.113543.6006.99.4513285645466080530,1401,80,4ch_sense,4CH,1.00
4,CHD1055302,2.16.124.113543.6006.99.4575318287899584458,701,120,4ch_sense,4CH,1.00
8,CHD1055302,2.16.124.113543.6006.99.7493418895009265798,1801,30,lt_pa_sense,SA,0.73


In [8]:
# exclude low confidence predictions
input_df = input_df.loc[(input_df['Confidence'] > 0.95)]
input_df

,Patient ID,Series ID,Series Number,Frames,Series Description,Predicted View,Confidence
1,CHD1055302,2.16.124.113543.6006.99.4545974342301296934,1101,480,sa_sense,SA,1.0
2,CHD1055302,2.16.124.113543.6006.99.4513285645466080530,1401,80,4ch_sense,4CH,1.0
4,CHD1055302,2.16.124.113543.6006.99.4575318287899584458,701,120,4ch_sense,4CH,1.0


#### Loading DICOM files for desired series

Now that we have identified some potential series, we can go ahead and load the dicom files for these series. The code below iterates over all the dicom files in the input directory, selecting each desired series.

In [12]:
src = '../data/example/CHD10553/' #UPDATE with correct patient information

print('Reading file list...')
unsortedList = []
for root, dirs, files in os.walk(src):
    for file in files: 
        if ".dcm" in file: # exclude non-dicoms, good for messy folders
            unsortedList.append(os.path.join(root, file))

print('%s files found.' % len(unsortedList))

Reading file list...
1172 files found.


In the following code, we will load the all of the images for the file list we just generated. This images will be placed into a pandas dataframe as a placeholder with the corresponding series and instance information. 

In [13]:
def clean_text(string):
    # clean and standardize text descriptions, which makes searching files easier
    forbidden_symbols = ["*", ".", ",", "\"", "\\", "/", "|", "[", "]", ":", ";", " "]
    for symbol in forbidden_symbols:
        string = string.replace(symbol, "_") # replace everything with an underscore
    
    return string.lower() 

def get_series_headers(dicom_list, series_list):
    # load dicom files for each series in a list of series IDs. 
    output = []
    
    for dicom_loc in tqdm(dicom_list):
        # read dicom file and return header information and image
        ds = pydicom.read_file(dicom_loc, force=True)
        seriesInstanceUID = ds.get("SeriesInstanceUID","NA")
        
        if seriesInstanceUID in series_list:
            # get patient, study, and series information
            patientID = clean_text(ds.get("PatientID", "NA"))
            seriesInstanceUID = ds.get("SeriesInstanceUID","NA")
            instanceNumber = str(ds.get("InstanceNumber","0"))

            # load image data
            array = ds.pixel_array
    
            output.append([patientID, dicom_loc, seriesInstanceUID, instanceNumber, array])
    
    return output

In [14]:
output = get_series_headers(unsortedList, list(input_df['Series ID']))

100%|█████████████████████████████████████████████████████████████████████████████| 1172/1172 [00:05<00:00, 223.65it/s]


In [15]:
out = pd.DataFrame(output, columns = ['Patient ID', 'Filepath', 'Series ID', 'Instance ID', 'Array'])

In [16]:
out

,Patient ID,Filepath,Series ID,Instance ID,Array
0,chd1055302,../data/example/CHD10553/CHD1055302\CAP_CHD105...,2.16.124.113543.6006.99.4513285645466080530,42,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,chd1055302,../data/example/CHD10553/CHD1055302\CAP_CHD105...,2.16.124.113543.6006.99.4513285645466080530,23,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,chd1055302,../data/example/CHD10553/CHD1055302\CAP_CHD105...,2.16.124.113543.6006.99.4513285645466080530,54,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,chd1055302,../data/example/CHD10553/CHD1055302\CAP_CHD105...,2.16.124.113543.6006.99.4513285645466080530,33,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,chd1055302,../data/example/CHD10553/CHD1055302\CAP_CHD105...,2.16.124.113543.6006.99.4575318287899584458,16,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...
675,chd1055302,../data/example/CHD10553/CHD1055302\CAP_CHD105...,2.16.124.113543.6006.99.4545974342301296934,383,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
676,chd1055302,../data/example/CHD10553/CHD1055302\CAP_CHD105...,2.16.124.113543.6006.99.4545974342301296934,410,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
677,chd1055302,../data/example/CHD10553/CHD1055302\CAP_CHD105...,2.16.124.113543.6006.99.4545974342301296934,51,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
678,chd1055302,../data/example/CHD10553/CHD1055302\CAP_CHD105...,2.16.124.113543.6006.99.4545974342301296934,361,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


#### Generating 2D + Time Volumes

Now that all of the images have been loaded, we need to format them into a 2D + time volume that the neural network can use as an input. The images are opened according to the window width and level specified in the DICOM headers. The frames are placed into the volume according to the instance number. The neural networks for this notebook were trained on series of the format (batch_size, 30, 224, 224, 3); as a result, only series with 30 frames per slice location will be included. The following functions will complete this task for us: 

In [17]:
# define windowing function 
def windowing(image, window_center, window_width):
    """Clip an array to the appropriate range given a window width and level"""
    # calculate min and max pixel values
    min_value = window_center - window_width / 2
    max_value = window_center + window_width / 2
    
    # clip values to appropriate window
    return np.clip(image, min_value, max_value)

In [18]:
def volume_from_instance_idx(series_dcm_list):
    """
    Loads a 3D dicom image volume with the phase + slice (or instances if phases are unavailable) defining each volume
    :param inpath: list of dicoms in each series
    :return: (slices, phases, 224, 224, 1) array
    """

    # save image dimensions, window, and tags from the first image in the list
    imshape = (224,224)
    window_center = series_dcm_list[0][0x0028, 0x1050].value
    window_width = series_dcm_list[0][0x0028, 0x1051].value
    tags = [series_dcm_list[0].SeriesNumber]
    
    # find the number of frames in the series (ideally from the dicom header)
    try:
        number_of_frames = series_dcm_list[0][0x0020, 0x1002].value
    except:
        try: 
            phases = series_dcm_list[0][0x2001, 0x1017].value
            slices = series_dcm_list[0][0x2001, 0x1018].value
            number_of_frames = phases * slices
        except:
            number_of_frames = len(series_dcm_list)
        
    if number_of_frames % 30 == 0:
        
        # store slice locations in list
        slice_locations = [dcm[0x0020, 0x1041].value for dcm in series_dcm_list]
        
        # make map to convert slice locations into consecutive integers
        map_dict = {}
        for i, loc in enumerate(sorted(list(set(slice_locations)))):
            map_dict[loc] = i
        
        # create a volume of the appropriate size
        vol = np.zeros((int(len(set(slice_locations))), 30, imshape[0], imshape[1], 1), dtype=np.uint16)
        
        reversed_map = False 
        
        # iterate through dcms and assign pixel_array to the appropriate location in the slice
        for dcm in series_dcm_list:
            slice_loc = dcm[0x0020, 0x1041].value
            slice_idx = map_dict[slice_loc]
            phase_idx = int(dcm.InstanceNumber - 30 * slice_idx) - 1         # instanceNumber - 30 * slice_iterator = phase location
            #print('Calculated phase index of {} and slice of {}'.format(phase_idx, slice_idx))
            
            if phase_idx > 30 or phase_idx < 0:
                
                # will need to reverse the order of slice locations
                map_dict = {}
                for i, loc in enumerate(reversed(sorted(list(set(slice_locations))))):
                    map_dict[loc] = i
                    
                reversed_map = True
                    
                # reiterate through dcms
                for dcm in series_dcm_list:
                    slice_loc = dcm[0x0020, 0x1041].value
                    slice_idx = map_dict[slice_loc]
                    phase_idx = int(dcm.InstanceNumber - 30 * slice_idx) - 1 
                    
                    img = windowing(dcm.pixel_array, window_center, window_width).astype(np.uint16)
                    resized = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
                    # insert into volume
                    vol[slice_idx, phase_idx, :, :, 0] = np.copy(resized)
            
            elif reversed_map == False:
                           
                img = windowing(dcm.pixel_array, window_center, window_width).astype(np.uint16)
                resized = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
                # insert into volume
                vol[slice_idx, phase_idx, :, :, 0] = np.copy(resized)
            
            else:
                break
            
        print('Created volume of size {}'.format(vol.shape))
        
        return vol     
     
    else:
        print("Series does not contain enough frames/phases: skipping series {}".format(tags))

### Input Generator

The trained neural networks expect an input of the size (batch_size, 30, 224, 224, 3). For a given batch, the input contains 30 time points of 3-channel images, where the channels are the t, t+1, and t+2 frames for a given series. To produce this input, we define a generator below that will accept the volumes generated above and produce the desired inputs. Furthermore, to help the networks reliably identify the correct end-systolic phase frame, regardless of which temporal position it occurs at in the series, the 30 frame series is "rolled" forward five times. For example, in a series where the ES phase occurs in the 8th frame as shown below: 

0, 1, 2, 3, 4, 5, 6, 7, 8 (ES Phase), 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,

will be rolled forward 4 times to also produce the following inputs: 

28, 29, 0, 1, 2, 3, 4, 5, 6, 7, 8 (ES Phase), 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 

26, 27, 28, 29, 0, 1, 2, 3, 4, 5, 6, 7, 8 (ES Phase), 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 

24, 25, 26, 27, 28, 29, 0, 1, 2, 3, 4, 5, 6, 7, 8 (ES Phase), 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 

22, 23, 24, 25, 26, 27, 28, 29, 0, 1, 2, 3, 4, 5, 6, 7, 8 (ES Phase), 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21. 


Ideally, the network will predict that the ES phase occurs at the 8th, 10th, 12th, 14th, and 16th index for the above inputs. These responses will then be rolled backwards and averaged to produce a final prediction. 
 

In [19]:
def generator(images, counter=0, window_size = 30):
    " Data generator for 2D + time images of shape (None, batch_sz, 30, 224, 224, 1)"
    keys = [x for x in range(images.shape[0])]
    
    while True:
        input_images = np.zeros((5, window_size, 224, 224, 3))
        #np.random.shuffle(keys)
        
        for i in range(5):
            # load images and concatenate along batch axis
            imgs = np.array(images[keys[counter], ...]).astype(np.float32)
            imgs = np.roll(imgs, i*2, 0)
            
            frame2 = np.roll(imgs, -1, 0) # get next 2 frames (t+1, t+2) to append to the original image
            frame3 = np.roll(imgs, -2, 0)
            imgs = tf.squeeze(np.stack((imgs, frame2, frame3), axis=3))
            
            imgs = np.expand_dims(imgs, 0)
            input_images[i] = np.concatenate(imgs, axis=0)
            
            # normalize images
            input_images[i] = (input_images[i] - np.min(input_images[i]))
            
            max_value = np.max(input_images[i])
            if max_value > 0:
                input_images[i] = (input_images[i] / max_value ) * 255.
            else:
                pass
            
            input_images[i] = tf.keras.applications.resnet50.preprocess_input(input_images[i])
        
        yield (input_images.astype(np.float32))
        counter += 1

#### Loading the Trained Model

The previously trained models include a ResNet5-LSTM network and a VGG19-LSTM network. These models can be loaded using the following code:

In [21]:
MODELPATH = '../models/phases/resnet50_lstm.hdf5'
model = tf.keras.models.load_model(MODELPATH)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 30, 7, 7, 2048)    23587712  
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 2048)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 30, 512)           5244928   
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 512)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 512)           2099200   
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 512)           0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 128)           6

#### Running the Complete Analysis

Now that we have defined the necessary functions, we can create the volumes for each 4CH and SA series that was found for the desired patient above and make a prediction of the ES phase using the trained neural network. 

In [22]:
volumes = []
if 'ES Phase Prediction' not in input_df.keys():
    input_df['ES Phase Prediction'] = ""
    
for series in out['Series ID'].unique():
    
    new = out.loc[out['Series ID'] == series]
    
    files = new['Filepath']
    dcm_list = []
    
    # iterate through files in current list
    for file in files:
        # check to ensure file is a valid dcm file
        try:
            dcm = pydicom.dcmread(file)
            dcm_list.append(dcm)

        except:
            # traceback.print_exc()
            pass
        
    vol = volume_from_instance_idx(dcm_list)
    
    if vol is not None: 
        pred_es = []
        for j in tqdm(range(vol.shape[0])):
            predictions = np.zeros((5,30))
            images = next(generator(vol, counter=j)) # for each volume, generate the corresponding input

            for i, img in enumerate(images):
                preds = model.predict_on_batch(tf.expand_dims(img,0))
                predictions[i,:] = np.roll(tf.squeeze(preds), -i*2, 0) # roll predictions backwards for each sequential input

            pred_es.append(np.argmax(np.mean(predictions, axis=0)))
            
        # add prediction to dataframe loaded from csv previously
        input_df.loc[input_df['Series ID'] == series, ['ES Phase Prediction']] = np.median(pred_es)

Series does not contain enough frames/phases: skipping series ["1401"]
Created volume of size (4, 30, 224, 224, 1)


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:08<00:00, 17.12s/it]


Created volume of size (16, 30, 224, 224, 1)


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [04:11<00:00, 15.73s/it]


The ES phase predictions have been added to the dataframe with the patient, series, and view information, as shown below:

In [23]:
input_df

,Patient ID,Series ID,Series Number,Frames,Series Description,Predicted View,Confidence,ES Phase Prediction
1,CHD1055302,2.16.124.113543.6006.99.4545974342301296934,1101,480,sa_sense,SA,1.0,13
2,CHD1055302,2.16.124.113543.6006.99.4513285645466080530,1401,80,4ch_sense,4CH,1.0,
4,CHD1055302,2.16.124.113543.6006.99.4575318287899584458,701,120,4ch_sense,4CH,1.0,13


In [24]:
# save this dataframe to a new csv file
input_df.to_csv(dst + '{}_phase_predictions.csv'.format(input_df.iloc[0]['Patient ID']))

### 2.0 Multiple Patients

To run the complete analysis over multiple patients, use the following code. To start, select the desired parameters.

In [25]:
#### PARAMETERS ####
csv_src = '../reports/EXAMPLE_series_predictions.csv'       # path to input csv containing view information
src = '../data/example/'                                # path to raw dicom files
dst = '../reports/'                                        # path to save resulting predictions

MODELPATH = '../models/phases/resnet50_lstm.hdf5'

If not already run above, define the necessary functions for the analysis by running the code block below. 

In [26]:
def clean_text(string):
    # clean and standardize text descriptions, which makes searching files easier
    forbidden_symbols = ["*", ".", ",", "\"", "\\", "/", "|", "[", "]", ":", ";", " "]
    for symbol in forbidden_symbols:
        string = string.replace(symbol, "_") # replace everything with an underscore
    
    return string.lower() 

def get_series_headers(dicom_list, series_list):
    # load dicom files for each series in a list of series IDs. 
    output = []
    
    for dicom_loc in dicom_list:
        # read dicom file and return header information and image
        ds = pydicom.read_file(dicom_loc, force=True)
        seriesInstanceUID = ds.get("SeriesInstanceUID","NA")
        
        if seriesInstanceUID in series_list:
            # get patient, study, and series information
            patientID = clean_text(ds.get("PatientID", "NA"))
            seriesInstanceUID = ds.get("SeriesInstanceUID","NA")
            instanceNumber = str(ds.get("InstanceNumber","0"))

            # load image data
            array = ds.pixel_array
    
            output.append([patientID, dicom_loc, seriesInstanceUID, instanceNumber, array])
    
    return output

def windowing(image, window_center, window_width):
    """Clip an array to the appropriate range given a window width and level"""
    # calculate min and max pixel values
    min_value = window_center - window_width / 2
    max_value = window_center + window_width / 2
    
    # clip values to appropriate window
    return np.clip(image, min_value, max_value)

def volume_from_instance_idx(series_dcm_list):
    """
    Loads a 3D dicom image volume with the phase + slice (or instances if phases are unavailable) defining each volume
    :param inpath: list of dicoms in each series
    :return: (slices, phases, 224, 224, 1) array
    """

    # save image dimensions, window, and tags from the first image in the list
    imshape = (224,224)
    window_center = series_dcm_list[0][0x0028, 0x1050].value
    window_width = series_dcm_list[0][0x0028, 0x1051].value
    tags = [series_dcm_list[0].SeriesNumber]
    
    # find the number of frames in the series (ideally from the dicom header)
    try:
        number_of_frames = series_dcm_list[0][0x0020, 0x1002].value
    except:
        try: 
            phases = series_dcm_list[0][0x2001, 0x1017].value
            slices = series_dcm_list[0][0x2001, 0x1018].value
            number_of_frames = phases * slices
        except:
            number_of_frames = len(series_dcm_list)
        
    if number_of_frames % 30 == 0: 
        
        # store slice locations in list
        slice_locations = [dcm[0x0020, 0x1041].value for dcm in series_dcm_list]
        
        # make map to convert slice locations into consecutive integers
        map_dict = {}
        for i, loc in enumerate(sorted(list(set(slice_locations)))):
            map_dict[loc] = i
        
        # create a volume of the appropriate size
        vol = np.zeros((int(len(set(slice_locations))), 30, imshape[0], imshape[1], 1), dtype=np.uint16)
        
        reversed_map = False 
        
        # iterate through dcms and assign pixel_array to the appropriate location in the slice
        for dcm in series_dcm_list:
            slice_loc = dcm[0x0020, 0x1041].value
            slice_idx = map_dict[slice_loc]
            phase_idx = int(dcm.InstanceNumber - 30 * slice_idx) - 1         # instanceNumber - 30 * slice_iterator = phase location
            #print('Calculated phase index of {} and slice of {}'.format(phase_idx, slice_idx))
            
            if phase_idx > 30 or phase_idx < 0:
                
                # will need to reverse the order of slice locations
                map_dict = {}
                for i, loc in enumerate(reversed(sorted(list(set(slice_locations))))):
                    map_dict[loc] = i
                    
                reversed_map = True
                    
                # reiterate through dcms
                for dcm in series_dcm_list:
                    slice_loc = dcm[0x0020, 0x1041].value
                    slice_idx = map_dict[slice_loc]
                    phase_idx = int(dcm.InstanceNumber - 30 * slice_idx) - 1 
                    
                    img = windowing(dcm.pixel_array, window_center, window_width).astype(np.uint16)
                    resized = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
                    # insert into volume
                    vol[slice_idx, phase_idx, :, :, 0] = np.copy(resized)
            
            elif reversed_map == False:
                img = windowing(dcm.pixel_array, window_center, window_width).astype(np.uint16)
                resized = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
                # insert into volume
                vol[slice_idx, phase_idx, :, :, 0] = np.copy(resized)
            
            else:
                break
        
        return vol     
     
    else:
        pass
        #print("Series does not contain enough frames/phases: skipping series {}".format(tags))
        
def generator(images, counter=0, window_size = 30):
    " Data generator for 2D + time images of shape (None, batch_sz, 30, 224, 224, 1)"
    keys = [x for x in range(images.shape[0])]
    
    while True:
        input_images = np.zeros((5, window_size, 224, 224, 3))
        #np.random.shuffle(keys)
        
        for i in range(5):
            # load images and concatenate along batch axis
            imgs = np.array(images[keys[counter], ...]).astype(np.float32)
            imgs = np.roll(imgs, i*2, 0)
            
            frame2 = np.roll(imgs, -1, 0) # get next 2 frames (t+1, t+2) to append to the original image
            frame3 = np.roll(imgs, -2, 0)
            imgs = tf.squeeze(np.stack((imgs, frame2, frame3), axis=3))
            
            imgs = np.expand_dims(imgs, 0)
            input_images[i] = np.concatenate(imgs, axis=0)
            
            # normalize images
            input_images[i] = (input_images[i] - np.min(input_images[i]))
            
            max_value = np.max(input_images[i])
            if max_value > 0:
                input_images[i] = (input_images[i] / max_value ) * 255.
            else:
                pass
            
            input_images[i] = tf.keras.applications.resnet50.preprocess_input(input_images[i])
        
        yield (input_images.astype(np.float32))
        counter += 1

Now that we have selected the parameters and defined the necessary functions, we can run the ES phase prediction analysis for every patient. 

Note - this script assumes that the patient ID (e.g., CHD1503301) can be found in the complete path to each of that specific patient's DICOM files. For example, either of the following directory structures would be acceptable: 

```bash
├── DATA
    └── PatientID
        ├── 1.dcm
        ├── 2.dcm 
        ├── 3.dcm          
        └── ...            
```

OR

```bash
├── DATA
    └── Subdir
        ├── *PatientID*.dcm
        ├── *PatientID*.dcm 
        ├── *PatientID*.dcm          
        └── ...            
```

In [27]:
# load the csv as a DataFrame using pandas
views = pd.read_csv(csv_src)
patients = views['Patient ID'].unique()
print('Available patients: {}'.format(patients))

print('Reading file list...')
unsortedList = []
for root, dirs, files in os.walk(src):
    for file in files: 
        if ".dcm" in file: # exclude non-dicoms, good for messy folders
            unsortedList.append(os.path.join(root, file))

# load desired model
print('Loading model from {}'.format(MODELPATH))
model = tf.keras.models.load_model(MODELPATH)

if 'ES Phase Prediction' not in views.keys():
    views['ES Phase Prediction'] = ""

print('Running ES phase prediction for all patients!')
for patient in tqdm(patients):
    patient_df = views.loc[views['Patient ID'] == patient]
    
    # select the 4CH and SA views for this patient
    input_df = patient_df.loc[(patient_df['Predicted View'].isin(['SA','4CH']))]
    
    # exclude low confidence predictions
    input_df = input_df.loc[(input_df['Confidence'] > 0.95)]
    
    # select dicom filenames corresponding to this patient
    patient_dicom_list = [file for file in unsortedList if patient in file]
    
    # load headers, arrays for each desired image
    output = get_series_headers(unsortedList, list(input_df['Series ID']))
    out = pd.DataFrame(output, columns = ['Patient ID', 'Filepath', 'Series ID', 'Instance ID', 'Array'])
    
    #
    volumes = []
    for series in out['Series ID'].unique():

        new = out.loc[out['Series ID'] == series]

        files = new['Filepath']
        dcm_list = []

        # iterate through files in current list
        for file in files:
            # check to ensure file is a valid dcm file
            try:
                dcm = pydicom.dcmread(file)
                dcm_list.append(dcm)

            except:
                # traceback.print_exc()
                pass

        vol = volume_from_instance_idx(dcm_list)

        if vol is not None: 
            pred_es = []
            for j in range(vol.shape[0]):
                predictions = np.zeros((5,30))
                images = next(generator(vol, counter=j)) # for each volume, generate the corresponding input

                for i, img in enumerate(images):
                    preds = model.predict_on_batch(tf.expand_dims(img,0))
                    predictions[i,:] = np.roll(tf.squeeze(preds), -i*2, 0) # roll predictions backwards for each sequential input

                pred_es.append(np.argmax(np.mean(predictions, axis=0)))

            # add prediction to dataframe loaded from csv previously
            views.loc[views['Series ID'] == series, ['ES Phase Prediction']] = np.median(pred_es)
            
views.to_csv(os.path.join(dst, 'ES_phase_predictions'))

Available patients: ['CHD1055302']
Reading file list...
Loading model from ../models/phases/resnet50_lstm.hdf5
Running ES phase prediction for all patients!


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [05:35<00:00, 335.83s/it]


In [28]:
# print out generated dataframe just to see results (also saved in csv)
views.loc[views['ES Phase Prediction'] != '']

,Patient ID,Series ID,Series Number,Frames,Series Description,Predicted View,Confidence,ES Phase Prediction
1,CHD1055302,2.16.124.113543.6006.99.4545974342301296934,1101,480,sa_sense,SA,1.0,13
4,CHD1055302,2.16.124.113543.6006.99.4575318287899584458,701,120,4ch_sense,4CH,1.0,13


In [29]:
reset()

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Don't know how to reset  (), please run `%reset?` for details
